In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from copy import deepcopy
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris

from sklearn.manifold import TSNE

In [4]:
iris = load_iris()
tsne = TSNE(n_components=2, random_state=0)
X = tsne.fit_transform(iris.data)

In [2]:
#importing the dataset
df=pd.read_csv("./s1.csv")
print("Input data & shape")
print(df.shape)
df.head()

Input data & shape
(5000, 2)


,v1,v2
0,664159,550946
1,665845,557965
2,597173,575538
3,618600,551446
4,635690,608046


In [3]:
df.head()

,v1,v2
0,664159,550946
1,665845,557965
2,597173,575538
3,618600,551446
4,635690,608046


In [5]:
#finding Kmax
kmax=int(sqrt(df.shape[0]))
kmax

12

In [6]:
#global centered
cen=np.array(np.mean(df))
print(cen)

0.14381672


In [7]:
# Euclidean Distance Caculator
def dist(a, b, ax=1):
    return np.linalg.norm(a - b, axis=ax)


In [7]:
# Getting the values and plotting it
f1 = df['v1'].values
f2 = df['v2'].values
X = np.array(list(zip(f1,f2)))

In [8]:
# calculate AvgDistance
def AvgDist(a):
    sum=0;
    for i in range(len(a)):
        for j in range(len(a)):
            if(j > i):
                #print(i,j)
                sum=sum+dist(a[i],a[j],None)
        #print(" ")
    #print("total sum is:",sum)
    AvgDist=(2/(len(a)*(len(a)-1)))*sum
    
    return AvgDist

In [9]:
#calculating epsilon neighbour value
epsilon=0.125*AvgDist(X)
print(epsilon)

2.5414933306965795


In [10]:
# calculating density parameter
def DensityParameter(a):
    S=[]
    
    for i in range(len(a)):
        total=0
        for j in range(len(a)):
            if(i != j):
                sum=epsilon - dist(a[i],a[j],None)
                if (sum >=0):
                    sum=1
                else:
                    sum=0
                total=total+sum
        S.append(total)
    #index_max = np.argmax(S)
    #print (S)
    return S#a[index_max]       
   

In [11]:
#Storing density paremeter value 
dpl=DensityParameter(X)
#print(final)

In [12]:
VCVI=np.zeros(kmax)

In [14]:
for k in range(2,kmax):
    final=dpl.copy()
    #creating an array for storing initial cenroid
    a=[]
    #finding point with highest density parameter value
    for i in range(k):
        index_max=np.argmax(final)
#        print("index number:", index_max)
#        print("original value at that index:",X[index_max])
        a.append(X[index_max])
        final[index_max]=-1
        for j in range(len(X)):
            if (dist(a[i],X[j],None) < epsilon):
                final[j]=-1
                
    #Initial centroids
    C=np.array(list(a),dtype=np.float32)
    print("Initial Centroids for k=",k)
    print(C)
    # To store the value of centroids when it updates
    C_old = np.zeros(C.shape)
#    print(C_old)
# Cluster Lables(0, 1, 2)
    clusters = np.zeros(len(X))
#    print(clusters)

# Error func. - Distance between new centroids and old centroids
    error=dist(C,C_old,None)
#    print(error)
# Loop will run till the error becomes zero

    while error != 0:
        # Assigning each value to its closest cluster
        for i in range(len(X)):
            distances = dist(X[i], C)
           # print("distance=",distances,"X[i]=",X[i],"C=",C)
            cluster = np.argmin(distances)
           # print("cluster=",cluster)
            clusters[i] = cluster
        # Storing the old centroid values
        C_old = deepcopy(C)
       # print("old centroid=",C_old)
    
        # Finding the new centroids by taking the average value
        for i in range(k):
            points = [X[j] for j in range(len(X)) if clusters[j] == i]
            C[i] = np.mean(points, axis=0)
        error = dist(C, C_old, None)
#    print("new centroid=",C)
#    count=count+1
#    print("count=",count)
    difference=0
    for i in range(k):
        difference=difference+dist(cen,C[i],None)
#    print("cen=",cen,"C[i]=",C[i])
#    print(difference)
    G=(difference/k)
    #calculating Ti
    total=0
    for i in range(k):
            points = np.array([X[j] for j in range(len(X)) if clusters[j] == i])
            for j in range(len(points)):
#                print("point=",points[j],"center=",C[i],"distance=",dist(points[j],C[i],None))
                total=total+dist(points[j],C[i],None)
#        print(" ")
    T=total/df.shape[0]
    VCVI[k]=((k*k*G)+((df.shape[0])*T))/df.shape[0]

Initial Centroids for k= 2
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]]
Initial Centroids for k= 3
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.800422 ]]
Initial Centroids for k= 4
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.800422 ]
 [ -4.4949164  -7.5312047]]
Initial Centroids for k= 5
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.800422 ]
 [ -4.4949164  -7.5312047]
 [ 12.435471   20.738504 ]]
Initial Centroids for k= 6
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.800422 ]
 [ -4.4949164  -7.5312047]
 [ 12.435471   20.738504 ]
 [ -5.3851194 -15.092701 ]]
Initial Centroids for k= 7
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.800422 ]
 [ -4.4949164  -7.5312047]
 [ 12.435471   20.738504 ]
 [ -5.3851194 -15.092701 ]
 [ 12.2397     25.997301 ]]
Initial Centroids for k= 8
[[ 12.394301   23.352915 ]
 [ -6.4698305 -17.44527  ]
 [ -5.8403697 -11.8004

In [15]:
VCVI

array([ 0.        ,  0.        ,  3.9617282 ,  3.30786691,  3.3850938 ,
        4.41779499,  5.63337386,  7.46645843,  8.92611038, 10.30625   ,
       11.84651381, 14.13866084])

In [16]:
min=VCVI[2]
min

3.961728200962146

In [17]:
for k in range(3,kmax):
    if(min > VCVI[k]):
        min=VCVI[k]
        kopt=k
    

In [18]:
print(kopt)
print(VCVI[kopt])

3
3.3078669121364754
